In [1]:
# -- MNISTとクラスの手書き文字を使用したニューラルネットワーク --

import torchvision.datasets as datasets
import torchvision.transforms as transforms

data_root = "./data"
data_root_mnist = f"{data_root}/mnist"

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.Lambda(lambda x: x.view(-1))
])

mnist_train_set = datasets.MNIST(
    root = data_root_mnist,
    train = True,
    download = True,
    transform = transform
)

mnist_test_set = datasets.MNIST(
    root = data_root_mnist,
    train = False,
    download = True,
    transform = transform
)

print(f"MNISTの学習データ件数: {len(mnist_train_set)}")
print(f"MNISTのテストデータ件数: {len(mnist_test_set)}")

MNISTの学習データ件数: 60000
MNISTのテストデータ件数: 10000


In [2]:
import shutil
import os
from glob import glob
import random
from PIL import Image

zip_root_hand_digits = "./hand_digits.zip"
data_root_hand_digits = f"{data_root}/hand_digits"
data_root_hand_digits_train = f"{data_root_hand_digits}/train"
data_root_hand_digits_test = f"{data_root_hand_digits}/test"
data_root_hand_digits_tmp = f"{data_root_hand_digits}/tmp"

shutil.unpack_archive(zip_root_hand_digits, data_root_hand_digits_tmp)

for n in range(10):
    no_dir = f"{data_root_hand_digits_train}/{n}"
    os.makedirs(no_dir, exist_ok=True)

for n in range(10):
    no_dir = f"{data_root_hand_digits_test}/{n}"
    os.makedirs(no_dir, exist_ok=True)

my_hand_digits_images = glob(f"{data_root_hand_digits_tmp}/*.png")

for file in my_hand_digits_images:
    with Image.open(file) as img:
        for angle in range(-45, 46, 5):
            if angle:
                img_r = img.rotate(angle, fillcolor=0xffffff)
                img_r.save(file.replace('.png', f'_A{angle}.png'))

my_hand_digits_images = glob(f'{data_root_hand_digits_tmp}/**/*.png', recursive=True)

print("クラスの手書き文字の画像ファイル数:", len(my_hand_digits_images))
random.seed(0)
random.shuffle(my_hand_digits_images)

train_size = int(0.8 * len(my_hand_digits_images))
test_size = len(my_hand_digits_images) - train_size

for file in my_hand_digits_images[:train_size]:
    file_name = os.path.basename(file)
    label = file_name[0]
    dest_dir = f"{data_root_hand_digits_train}/{label}"
    shutil.move(file, f"{dest_dir}/{file_name}")

for file in my_hand_digits_images[train_size:]:
    file_name = os.path.basename(file)
    label = file_name[0]
    dest_dir = f"{data_root_hand_digits_test}/{label}"
    shutil.move(file, f"{dest_dir}/{file_name}")

shutil.rmtree(data_root_hand_digits_tmp)

クラスの手書き文字の画像ファイル数: 18962


In [5]:
hand_digits_train_set = datasets.ImageFolder(
    root = data_root_hand_digits_train,
    transform = transform
)

hand_digits_test_set = datasets.ImageFolder(
    root = data_root_hand_digits_test,
    transform = transform
)

In [6]:
from torch.utils.data import ConcatDataset

train_set = ConcatDataset([mnist_train_set, hand_digits_train_set])
test_set = ConcatDataset([mnist_test_set, hand_digits_test_set])

print(f"統合した学習データ件数: {len(train_set)}")
print(f"統合したテストデータ件数: {len(test_set)}")

統合した学習データ件数: 75169
統合したテストデータ件数: 13793


In [7]:
from torch.utils.data import DataLoader

batch_size = 100

train_loader = DataLoader(
    train_set, 
    batch_size = batch_size,
    shuffle = True
)

test_loader = DataLoader(
    test_set,  
    batch_size = batch_size,
    shuffle = True
)

In [8]:
import torch.nn as nn

class Net2(nn.Module):
    def __init__(self, n_input, n_output, n_hidden):
        super().__init__()

        self.l1 = nn.Linear(n_input, n_hidden)
        self.l2 = nn.Linear(n_hidden, n_hidden)
        self.l3 = nn.Linear(n_hidden, n_output)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x1 = self.l1(x)
        x2 = self.relu(x1)
        x3 = self.l2(x2)
        x4 = self.relu(x3)
        x5 = self.l3(x4)
        return x5

In [9]:
import torch
import numpy as np

inputs, labels = next(iter(train_loader))

n_input = inputs[0].shape[0]
n_output = 10
n_hidden = 128

torch.manual_seed(123)
torch.cuda.manual_seed(123)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

net = Net2(
    n_input = n_input,
    n_output = n_output,
    n_hidden = n_hidden
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net.to(device)

leaning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=leaning_rate)
num_epochs = 50
history = np.zeros((0, 5))

In [10]:
from tqdm import tqdm

for epoch in range(num_epochs):
    n_train_acc, n_val_acc = 0, 0
    train_loss, val_loss = 0, 0
    n_train, n_val = 0, 0

    for inputs, labels in tqdm(train_loader):
        train_batch_size = len(labels)
        n_train += train_batch_size

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        predicted = torch.max(outputs, 1)[1]

        train_loss += loss.item() * train_batch_size
        n_train_acc += (predicted == labels).sum().item()

    for inputs_test, labels_test in test_loader:
        test_batch_size = len(labels_test)
        n_val += test_batch_size

        inputs_test = inputs_test.to(device)
        labels_test = labels_test.to(device)

        outputs_test = net(inputs_test)

        loss_test = criterion(outputs_test, labels_test)

        predicted_test = torch.max(outputs_test, 1)[1]

        val_loss += loss_test.item() * test_batch_size
        n_val_acc += (predicted_test == labels_test).sum().item()

    train_acc = n_train_acc / n_train
    val_acc = n_val_acc / n_val
    ave_train_loss = train_loss / n_train
    ave_val_loss = val_loss / n_val

    print (f'Epoch [{epoch + 1}/{num_epochs}], loss: {ave_train_loss:.5f} acc: {train_acc:.5f} val_loss: {ave_val_loss:.5f}, val_acc: {val_acc:.5f}')

    item = np.array([epoch + 1, ave_train_loss, train_acc, ave_val_loss, val_acc])
    history = np.vstack((history, item))

100%|██████████| 752/752 [00:37<00:00, 20.05it/s]


Epoch [1/50], loss: 1.49602 acc: 0.55490 val_loss: 0.99942, val_acc: 0.70130


100%|██████████| 752/752 [01:16<00:00,  9.88it/s]


Epoch [2/50], loss: 0.77708 acc: 0.77986 val_loss: 0.78891, val_acc: 0.79381


100%|██████████| 752/752 [01:43<00:00,  7.23it/s]


...

Epoch [35/50], loss: 0.09570 acc: 0.97363 val_loss: 0.13213, val_acc: 0.95860


100%|██████████| 752/752 [00:45<00:00, 16.55it/s]


Epoch [36/50], loss: 0.09238 acc: 0.97482 val_loss: 0.12833, val_acc: 0.95976


 54%|█████▎    | 404/752 [00:26<00:22, 15.26it/s]


KeyboardInterrupt: 